In [1]:
%load_ext autoreload

%autoreload 2

%matplotlib inline

In [4]:
import pandas as pd

from library import *

In [5]:

host = '10.0.5.225'
db = 'billin'
user = 'postgres'
password = 'ppoSgre123'

db = DB(user=user, password=password, host=host, db=db)

In [18]:
paymets_methods = db.gettable('f_invoice_payments')
paymets_methods.set_index('id_invoices', inplace=True)

In [19]:
paymets_methods.provider.unique()
paymets_methods.state.unique()

array(['gocardless', 'stripe', 'unnax'], dtype=object)

In [22]:
paymets_methods.head()

,uuid_payment_id,payment_created_at,payment_updated_at,amount,provider,state,charge_id,transfer_id,sincro_at,id_business
id_invoices,,,,,,,,,,
59f7546e3d496965e9c18b66,5a47562e6fb891690fe2b94f,20171230,20180109,59.29,gocardless,paid_out,PM001M4TEWBJHR,59f7546e3d496965e9c18b66,2018-07-12 21:54:52.567036+00:00,-1
59f7568e388b47657851c58b,5a4756406fb891690fe2b952,20171230,20180109,36.30,gocardless,paid_out,PM001M4TEYR5FF,59f7568e388b47657851c58b,2018-07-12 21:54:52.567036+00:00,53300
5a6cd1f039d60273ecf04f42,5a6cd31186e2c2373c1d3526,20180127,20180127,3.00,stripe,transfer,ch_1BozM5JahFDQqyZkgYN5WvuW,tr_1BozM7JahFDQqyZke7r45WMA,2018-07-12 21:54:52.567036+00:00,53305
5a6ccf6de4c1f8750a151f40,5a6cd496e4c1f8750a151f4f,20180127,20180127,3.00,stripe,transfer,ch_1BozSMJahFDQqyZkHjEedee9,tr_1BozSPJahFDQqyZk5MHZ9jnH,2018-07-12 21:54:52.567036+00:00,53305
5a6ce3e3e4c1f8750a151f58,5a6ce4b927fdb473f149f41b,20180127,20180127,3.00,stripe,succeeded,ch_1Bp0X0JahFDQqyZk2S0smHcd,tr_1Bp0X2JahFDQqyZkWgvnmtvQ,2018-07-12 21:54:52.567036+00:00,53305


In [23]:
provider = pd.get_dummies(paymets_methods.provider)

In [26]:
state = pd.get_dummies(paymets_methods.state)

In [36]:
paymets_methods.payment_updated_at = pd.to_datetime(paymets_methods.payment_updated_at, format='%Y%m%d')
paymets_methods.payment_created_at = pd.to_datetime(paymets_methods.payment_created_at, format='%Y%m%d')

In [38]:
payment_delay = paymets_methods.payment_updated_at - paymets_methods.payment_created_at

In [111]:
amount_dict = dict()
for b in paymets_methods.id_business.unique().tolist():
    amount_dict[b] = {}
    tempdf = paymets_methods[paymets_methods.id_business == b]
    amount_dict[b]['mean'] = tempdf.amount.mean()
    amount_dict[b]['std'] = np.nan_to_num(tempdf.amount.std())
    amount_dict[b]['max'] = np.nan_to_num(tempdf.amount.max())
    amount_dict[b]['min'] = np.nan_to_num(tempdf.amount.min())
    amount_dict[b]['count'] = tempdf.amount.count()
    
    payment_delay = tempdf.payment_updated_at - tempdf.payment_created_at
    amount_dict[b]['payment_delay_mean'] = payment_delay.mean()
    amount_dict[b]['payment_delay_std'] = payment_delay.std()
    amount_dict[b]['payment_delay_max'] = payment_delay.max()
    amount_dict[b]['payment_delay_min'] = payment_delay.min()
    
    for p in tempdf.provider.unique().tolist():
        temp_p = tempdf[tempdf.provider == p]
        amount_dict[b][p] = int(temp_p.shape[0])
    


In [120]:
amount = pd.DataFrame.from_dict(amount_dict,orient='index')
amount = amount.fillna(0)
amount[['gocardless','stripe','unnax']] = amount[['gocardless','stripe','unnax']].astype(int)

In [121]:
amount

,mean,std,max,min,count,payment_delay_mean,payment_delay_std,payment_delay_max,payment_delay_min,gocardless,stripe,unnax
-1,105.179019,1.430717e+02,665.830000,1.140000,35,6 days 13:01:42.857142,6 days 17:39:46.636942,33 days,0 days,34,1,0
0,66.048277,9.205018e+01,474.320000,18.150000,31,19 days 08:30:58.064516,20 days 23:52:06.574179,73 days,0 days,31,0,0
51936,115.689975,1.316940e+02,406.560000,16.456000,8,5 days 09:00:00,3 days 08:02:59.943785,8 days,0 days,8,0,0
52688,2.000000,0.000000e+00,2.000000,2.000000,1,4 days 00:00:00,0 days 00:00:00,4 days,4 days,1,0,0
52712,2.420000,0.000000e+00,2.420000,2.420000,1,0 days 00:00:00,0 days 00:00:00,0 days,0 days,0,1,0
52731,1650.004400,0.000000e+00,1650.004400,1650.004400,1,0 days 00:00:00,0 days 00:00:00,0 days,0 days,0,1,0
53300,119.428052,2.860040e+02,2420.000000,21.780000,77,8 days 14:57:39.740259,2 days 07:44:56.657892,14 days,0 days,77,0,0
53305,3.940816,5.991304e+00,60.000000,3.000000,147,0 days 00:00:00,0 days 00:00:00,0 days,0 days,0,147,0
57613,1006.516364,3.550383e+02,1311.000000,469.680000,11,7 days 04:21:49.090909,2 days 18:52:18.935174,12 days,0 days,10,1,0
57674,79.000000,0.000000e+00,79.000000,79.000000,4,0 days 00:00:00,0 days 00:00:00,0 days,0 days,0,4,0
